# Retail video analytics pipeline that tracks customer paths


1. Detect & track people

We use YOLO to track humans.

Each person gets:

Unique ID
Movement trajectory

2. Heatmap generation

We accumulate positions of people.

More visits = hotter area.

This shows:

  Popular aisles

3. Loitering detection

We define a:

  1. High-value zone (rectangle)

If a person stays there longer than X seconds:

  2.  Flag as loitering

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving retail.mp4 to retail.mp4


In [ ]:
!pip install ultralytics opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model = YOLO("yolov8n.pt")

# Open retail video
cap = cv2.VideoCapture("retail.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("retail_output.mp4", fourcc, fps,
                      (frame_width, frame_height))

# Heatmap accumulator
heatmap = np.zeros((frame_height, frame_width), dtype=np.float32)

# High-value zone (example rectangle)
zone_x1, zone_y1 = 200, 150
zone_x2, zone_y2 = 450, 350

# Loitering tracking
loiter_time = {}
loiter_threshold = fps * 3  # 3 seconds

while True:
    ret, frame = cap.read()
    if not ret:
        break

    overlay = frame.copy()

    # Draw high-value zone
    cv2.rectangle(frame, (zone_x1, zone_y1),
                  (zone_x2, zone_y2), (255, 0, 0), 2)

    # YOLO tracking (people only)
    results = model.track(frame, persist=True)[0]

    if results.boxes is not None and results.boxes.id is not None:

        boxes = results.boxes.xyxy.cpu().numpy()
        ids = results.boxes.id.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()

        for box, pid, cls in zip(boxes, ids, classes):

            # Class 0 = person
            if int(cls) != 0:
                continue

            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            # Update heatmap
            heatmap[cy, cx] += 1

            # Draw person box
            cv2.rectangle(frame, (x1, y1),
                          (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID {int(pid)}",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 255), 2)

            # Check loitering
            inside_zone = (zone_x1 < cx < zone_x2 and
                           zone_y1 < cy < zone_y2)

            if inside_zone:
                loiter_time[pid] = loiter_time.get(pid, 0) + 1
            else:
                loiter_time[pid] = 0

            if loiter_time.get(pid, 0) > loiter_threshold:
                cv2.putText(frame, "LOITERING",
                            (x1, y2 + 25),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.8, (0, 0, 255), 2)

    # Create heatmap overlay
    heatmap_norm = cv2.normalize(
        heatmap, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_color = cv2.applyColorMap(
        heatmap_norm.astype(np.uint8),
        cv2.COLORMAP_JET)

    blended = cv2.addWeighted(frame, 0.7,
                              heatmap_color, 0.3, 0)

    out.write(blended)

cap.release()
out.release()

print("Retail analytics complete! Saved as retail_output.mp4")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 221ms
Prepared 1 package in 855ms
Installed 1 package in 2ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 1.5s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 10 persons, 265.3ms
Speed: 14.3ms preprocess, 265.3ms inference, 40.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 96.2ms
Speed: 1.9ms preprocess, 96.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 99.9ms
Speed: 2.0ms preprocess, 99.9ms inferenc

In [ ]:
from google.colab import files
files.download("retail_output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>